# Proyecto de análisis de sentimientos con Python

##### Carmen Giles Floriano

En este proyecto se realizará un modelo de aprendizaje automático capaz de analizar tweets y predecir el sentimiento del usuario dentro de las siguientes categorías: "Muy feliz", "Contento", "Neutro", "Molesto" y "Hater". 

#### Ejercicio 1. RECOPILACIÓN DE DATOS

In [55]:
import pandas 
#import numpy
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob


En primer lugar, se lee el fichero `fifa_tweets_emotion.csv` mediante el paquete pandas. Los datos fueron obtenidos de Kaggle y son un conjunto de tweets relacionados con la WorldCup del FIFA 2022. 

In [66]:
# Con la función "read_csv" del paquete "pandas", leemos el data set indicando los nombres de las columnas.
tweet_fifa_prev = pandas.read_csv('fifa_tweets_emotion.csv', header=None,
                       names=['number', 'date', 'number_likes','source', 'tweet','feeling'])
print(tweet_fifa_prev.shape)
tweet_fifa_prev.head(5)

(22525, 6)


,number,date,number_likes,source,tweet,feeling
0,NaN,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
1,0.0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
2,1.0,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
3,2.0,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
4,3.0,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive


El data set presenta 22525 tweets distintos, cada uno representado en una fila. Además, del texto del propio tweet existen otras columnas con más información, como la fecha de publicación, el número de likes de la publicación, el lugar desde dónde se publicó y el sentimiento asociado. 

En este proyecto, sólo nos interesa el texto correspondiente al tweet. Los demás datos son eliminados, incluso la columna correspondiente a los sentimientos, ya que estos serán asignados siguiendo el criterio de la herramienta TextBlob. 

In [3]:
# Con la función "drop" se eliminan elementos del data set.
## En la lista "delete_col" añadimos los nombres de las columnas que deseamos eliminar, la cuál será un parámetro de la función
## "drop". Además, a esta función se le da el parámetro axis=1, lo que indica que los elementos a eliminar son columnas.

delete_col=["number", "date", "number_likes", "source", "feeling"]
tweet_fifa_col= tweet_fifa_prev.drop(delete_col, axis=1)

## Por otro lado, eliminamos la primera fila que contiene los anteriores nombres de las columnas del data set. Se elimina 
## indicando 0 (primer elemento) y el parámetro axis=0 (fila). 

tweet_fifa=tweet_fifa_col.drop(0, axis=0)
tweet_fifa.head(5)


,tweet
1,What are we drinking today @TucanTribe \n@MadB...
2,Amazing @CanadaSoccerEN #WorldCup2022 launch ...
3,Worth reading while watching #WorldCup2022 htt...
4,Golden Maknae shinning bright\n\nhttps://t.co/...
5,"If the BBC cares so much about human rights, h..."


#### Ejercicio 2. LIMPIEZA DEL TEXTO, ELIMINAR LAS PALABRAS QUE NO APORTAN INFORMACIÓN.

A continuación, procesamos el texto del tweet de forma que sea más fácil de codificar y analizar en pasos posteriores. Para ello se crea la función "limpiar_texto", que elimina elementos como menciones, hashtags, URLs y emoticonos; convierte el texto en minúscula; elimina las palabras poco informativas; y, además, realizará una lematización, es decir, se transformarán las palabras a su forma base.

In [46]:
## Recibe como entrada un data set
def limpiar_texto(data_set):
    ## En primer lugar, se cargan una serie de datos necesarios para el posterior procesamiento:
    # Se cargan los datos de emoticonos con el paquete "re" e indicando el código, unicode, del bloque de emoticonos 
    # correspondiente. El parámetro "flags=re.UNICODE" indica que la información añadida se lea según las reglas de unicode,es 
    # decir, para que sean identificados como emoticonos.
    patron_emoticonos = re.compile("["
                            u"\U0001F600-\U0001F64F"  # Emoticonos generales
                            u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                            u"\U0001F680-\U0001F6FF"  # Transporte y mapas
                            u"\U0001F780-\U0001F1FF"  # Formas geométricas extendidas
                            u"\U0001F900-\U0001F9FF"  # Emoticonos de personas y cuerpos
                            u"\U0001FA00-\U0001FA6F"  # Símbolos de objetos
                            u"\U0001FA70-\U0001FAFF"  # Símbolos de alimentos
                            "]+", flags=re.UNICODE)
    tokenizer=TweetTokenizer()
    nltk.download("stopwords")
    stopwords_english = stopwords.words("english")
    stemmer = SnowballStemmer("english")
    diccionario = {"tweet": []}
    
    ## Para cada fila del data set 
    for index, fila in data_set.iterrows():
        
        ## En la variable fila se tiene la información de la fila completa, al seleccionar "tweet", nos quedamos sólo con la 
        ## información de esa columna, es decir el texto del tweet correspondiente.
        tweet=fila["tweet"]
        
        ## Con la función "sub" del paquete "re", se eliminan una serie de elementos indicados en el primer parámetro de la 
        ## función (r'elemento'). 
        # Se eliminan los hashtags (#)
        tweet = re.sub(r'#', '', tweet)
        # Se eliminan las menciones, las cuáles tienen la siguiente estructura: @usuario. Al añadie \S+, se indica que también 
        #se elimina cualquier texto que esté inmediatamente después del elemento determinado, sin incluir espacios.
        tweet = re.sub(r'@\S+', '', tweet)
        # Se eliminan los URLs, que por lo general comienzan con http.
        tweet = re.sub(r'http\S+', '', tweet) 
        # Se eliminan los emoticonos usando el conjunto de emoticonos guardados, anteriormente, en "patrón_emoticonos".
        tweet = patron_emoticonos.sub(r'',tweet)
        
        # Poner en minuscula
        tweet=tweet.lower()
        
        tweet= tokenizer.tokenize(tweet)
        list_tweet=[]
        for palabra in tweet:
            if palabra not in stopwords_english:
                palabra_proc = stemmer.stem(palabra)
                list_tweet.append(palabra_proc)
        
        cadena_tweet=""
        for palabra in list_tweet:
            cadena_tweet+=" "
            cadena_tweet+=palabra
            
        diccionario["tweet"].append(cadena_tweet)
    data_set_proc=pandas.DataFrame(diccionario)
    return data_set_proc

In [47]:
tweet_fifa_limp=limpiar_texto(tweet_fifa)
tweet_fifa_limp.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet
0,drink today worldcup 2022
1,amaz worldcup 2022 launch video . show much f...
2,worth read watch worldcup 2022
3,golden makna shin bright jeonjungkook jungkoo...
4,"bbc care much human right , homosexu right , ..."


#### Ejercicio 3. ETIQUETADO DE DATOS CON HERRAMIENTAS YA EXISTENTES

In [48]:
def clasificador(data_set):
    diccionario={"tweet":[], "sentimiento":[]}
    for index, fila in data_set.iterrows():
        tweet=fila["tweet"]
        texto=TextBlob(tweet)
        sentimiento_pol=texto.sentiment.polarity
        if sentimiento_pol < (-0.6):
            sentimiento="Hater"
        elif (-0.6) <= sentimiento_pol < (-0.2):
            sentimiento="Molesto"
        elif (-0.2) <= sentimiento_pol < 0.2:
            sentimiento="Neutro"
        elif 0.2 <= sentimiento_pol < 0.6:
            sentimiento="Contento"
        else:
            sentimiento="Muy feliz"
        diccionario["tweet"].append(tweet)
        diccionario["sentimiento"].append(sentimiento)
    data_set_sent=pandas.DataFrame(diccionario)
    return data_set_sent

In [49]:
tweet_fifa_sent=clasificador(tweet_fifa_limp)
tweet_fifa_sent.head(5)

,tweet,sentimiento
0,drink today worldcup 2022,Neutro
1,amaz worldcup 2022 launch video . show much f...,Neutro
2,worth read watch worldcup 2022,Contento
3,golden makna shin bright jeonjungkook jungkoo...,Contento
4,"bbc care much human right , homosexu right , ...",Neutro


#### Ejercicio 4. CODIFICACIÓN DE LOS ATRIBUTOS Y OBJETIVOS.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [65]:
tweets=tweet_fifa_sent["tweet"]
tokens_list=[]
for tweet in tweets:
    tokens = word_tokenize(tweet)
    tokens_list.append(tokens)
    
# Entrenamiento de Word2Vec
model = Word2Vec(tokens_list, vector_size=100, window=5, min_count=1, workers=4)

# Obtención del embedding de una palabra

atributo={"tweet_vector":[]}
for tweet in tokens_list:
    vector=[]
    for palabra in tweet:
        if palabra in model.wv:
            embedding = model.wv[palabra]
            vector.append(embedding)
    if vector:
        media_vector=sum(vector)/len(vector)
        atributo["tweet_vector"].append(media_vector)
    else:
        atributo["tweet_vector"].append([])
atributo=pandas.DataFrame(atributo)

objetivo=tweet_fifa_sent["sentimiento"]

(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
       atributo, objetivo,
       random_state=12345,
       test_size=.2,
       stratify=objetivo)


La precisión del modelo Random Forest es 71.12097669256381


In [61]:
atributos_1=tweet_fifa_sent["tweet"]
vectorizer = CountVectorizer()
atributos_1 = vectorizer.fit_transform(atributos_1)

#Extrae el objetivo (ya es numérico)
objetivo_1= tweet_fifa_sent["sentimiento"]
#Separa conjunto de entrenamiento y de prueba
(atributos_entrenamiento_1, atributos_prueba_1,
 objetivo_entrenamiento_1, objetivo_prueba_1) = train_test_split(
        atributos_1, objetivo_1,
        random_state=12345,
        test_size=.2,
        stratify=objetivo)


La precisión del modelo desarrollado es 71.00998890122086


In [ ]:
#### RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(list(atributos_entrenamiento["tweet_vector"]), objetivo_entrenamiento)

# Realizar predicciones con el conjunto de prueba
predicciones_rf = rf_classifier.predict(list(atributos_prueba["tweet_vector"]))

# Calcular la precisión del modelo
precision_rf = np.mean(predicciones_rf == objetivo_prueba)
print("La precisión del modelo Random Forest es", precision_rf * 100)

In [ ]:
#### NAIVE BAYES
from sklearn.naive_bayes import MultinomialNB

#Entrena el modelo de Naive Bayes usando la instancia MultinomialNB que es recomendada 
#para este tipo de tareas
emotion_detector = MultinomialNB(alpha=1.0)  # alpha es el parámetro de suavizado
emotion_detector.fit(atributos_entrenamiento_1, objetivo_entrenamiento_1)

#Realiza las predicciones con el conjunto de prueba
predicciones = emotion_detector.predict(atributos_prueba_1)
#Calcular la precisión del modelo
precision = emotion_detector.score(atributos_prueba_1, objetivo_prueba_1)
print("La precisión del modelo Naive Bayes desarrollado es", precision*100)